In [1]:
from pyspark.sql import SparkSession

In [2]:
spark= SparkSession.builder.appName('SQL Operations').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/08 16:50:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/08 16:50:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/01/08 16:50:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/01/08 16:50:31 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/01/08 16:50:31 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/01/08 16:50:31 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


# Creating DataFrames and Registering Temporary Views

In [3]:
data = [("Alice", 23), ("Bob", 31), ("Cathy", 27)]
columns = ["Name", "Age"]

df = spark.createDataFrame(data, columns)

df.createOrReplaceTempView("people")

#  Querying Data with SQL Syntax

Once a DataFrame is registered as a view, you can query it using SQL. The spark.sql() function runs the SQL command and returns the result as a new DataFrame.

In [4]:
result_df = spark.sql("SELECT Name, Age FROM people WHERE Age > 25")
result_df.show()

+-----+---+
| Name|Age|
+-----+---+
|  Bob| 31|
|Cathy| 27|
+-----+---+



# DataFrame API and SQL Operations Together


In [5]:
result_df = spark.sql("SELECT Name, Age FROM people WHERE Age > 25")
result_transformed = result_df.withColumn("AgePlusOne", result_df["Age"] + 1)
result_transformed.show()

+-----+---+----------+
| Name|Age|AgePlusOne|
+-----+---+----------+
|  Bob| 31|        32|
|Cathy| 27|        28|
+-----+---+----------+



# Joining DataFrames Using SQL

In [6]:
orders_data = [(1, "2024-01-01", 150), (2, "2024-01-02", 200)]
orders_columns = ["OrderID", "OrderDate", "Amount"]
orders_df = spark.createDataFrame(orders_data, orders_columns)
orders_df.createOrReplaceTempView("orders")

customers_data = [(1, "Alice"), (2, "Bob")]
customers_columns = ["CustomerID", "CustomerName"]
customers_df = spark.createDataFrame(customers_data, customers_columns)
customers_df.createOrReplaceTempView("customers")

# SQL join operation
joined_df = spark.sql("""
    SELECT c.CustomerName, o.OrderDate, o.Amount 
    FROM customers c 
    JOIN orders o 
    ON c.CustomerID = o.OrderID
""")
joined_df.show()

+------------+----------+------+
|CustomerName| OrderDate|Amount|
+------------+----------+------+
|       Alice|2024-01-01|   150|
|         Bob|2024-01-02|   200|
+------------+----------+------+



# Aggregation and Grouping in SQL


In [7]:
agg_df = spark.sql("""
    SELECT CustomerName, COUNT(*) AS TotalOrders, SUM(Amount) AS TotalAmount
    FROM orders o 
    JOIN customers c ON o.OrderID = c.CustomerID
    GROUP BY CustomerName
""")
agg_df.show()

+------------+-----------+-----------+
|CustomerName|TotalOrders|TotalAmount|
+------------+-----------+-----------+
|         Bob|          1|        200|
|       Alice|          1|        150|
+------------+-----------+-----------+



#  Using Window Functions in PySpark SQL

In [8]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Define a window
windowSpec = Window.orderBy("Amount")

# Use row_number as a window function
result = orders_df.withColumn("RowNumber", row_number().over(windowSpec))
result.show()


25/01/08 16:50:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/08 16:50:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/08 16:50:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+----------+------+---------+
|OrderID| OrderDate|Amount|RowNumber|
+-------+----------+------+---------+
|      1|2024-01-01|   150|        1|
|      2|2024-01-02|   200|        2|
+-------+----------+------+---------+



# Advanced Transformations with PySpark SQL and DataFrame API

In [9]:

filtered_df = spark.sql("SELECT * FROM orders WHERE Amount > 100")
aggregated_df = filtered_df.groupBy("OrderID").sum("Amount")
final_df = aggregated_df.withColumnRenamed("sum(Amount)", "TotalAmount")
final_df.show()


+-------+-----------+
|OrderID|TotalAmount|
+-------+-----------+
|      1|        150|
|      2|        200|
+-------+-----------+



25/01/08 16:50:43 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
